In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

ds = xr.open_dataset("test_era5_t2m_20250101_0000.nc")

print(ds)

# Variable de température = t2m
var_name = "t2m"

# Sélectionner le premier instant (dim = valid_time)
da = ds[var_name].isel(valid_time=0)



In [ ]:
# Convertir en °C (Kelvin)
if hasattr(ds[var_name], "units") and ds[var_name].units in ["K", "kelvin"]:
    da = da - 273.15
    units = "°C"
else:
    units = getattr(ds[var_name], "units", "")

print(ds[var_name])


In [ ]:
print(da)

In [ ]:
plt.figure()
da.plot()
plt.title(f"Température 2m (valid_time=0) [{units}]")
plt.show()

In [ ]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point



In [ ]:
import fiona
print(fiona.listlayers("ADMIN-EXPRESS_4-0__GPKG_LAMB93_FXX_2025-11-20/ADMIN-EXPRESS/1_DONNEES_LIVRAISON_2025-11-00136/ADE_4-0_GPKG_LAMB93_FXX-ED2025-11-20/ADE_4-0_GPKG_LAMB93_FXX-ED2025-11-20.gpkg"))


# ATTENTION NE PAS LANCER LE CODE EN DESSOUS C'EST POUR LA CREATION DU SHX, ET C'EST DEJA FAIT

In [ ]:
from pathlib import Path

# Path to your GeoPackage
gpkg_file = Path("ADMIN-EXPRESS_4-0__GPKG_LAMB93_FXX_2025-11-20/ADMIN-EXPRESS/1_DONNEES_LIVRAISON_2025-11-00136/ADE_4-0_GPKG_LAMB93_FXX-ED2025-11-20/ADE_4-0_GPKG_LAMB93_FXX-ED2025-11-20.gpkg")

# Output folder
output_folder = Path("daaata")
output_folder.mkdir(parents=True, exist_ok=True)

# List all layers in the GeoPackage
layers = fiona.listlayers(gpkg_file)
print(f"Found layers: {layers}")

# Loop through each layer and save it as a separate shapefile
for layer in layers:
    print(f"Processing layer: {layer}")
    gdf = gpd.read_file(gpkg_file, layer=layer)
    output_file = output_folder / f"{layer}.shp"
    gdf.to_file(output_file)
    print(f"Saved: {output_file}")

print("All layers have been extracted successfully!")


In [ ]:
lats = ds["latitude"].values   # shape (43,)
lons = ds["longitude"].values  # shape (62,)

france = gpd.read_file("daaata/region.shx")


# S'assurer qu'on est en WGS84 (lat/lon degrees)
gdf = france.to_crs("EPSG:4326")

# Union de tous les polygones France (métropole + éventuels DOM)
france_poly = gdf.unary_union

# 3. Construire la grille des points centres des pixels
lon2d, lat2d = np.meshgrid(lons, lats)  # lon: axe x, lat: axe y

# Aplatir pour tester en bloc
points = [Point(x, y) for x, y in zip(lon2d.ravel(), lat2d.ravel())]

# 4. Test point-in-polygon pour chaque point
mask_flat = np.array([france_poly.contains(p) for p in points])
mask = mask_flat.reshape(lat2d.shape)  # shape (latitude, longitude)

# 5. Appliquer le masque à la température
t2m = ds["t2m"]  # (valid_time, latitude, longitude)
t2m_fr = t2m.where(mask)

da = t2m_fr.isel(valid_time=0)
if hasattr(ds[var_name], "units") and ds[var_name].units in ["K", "kelvin"]:
    da = da - 273.15
    units = "°C"
else:
    units = getattr(ds[var_name], "units", "")
plt.figure()
da.plot()
plt.title("T2m masquée sur la France")
plt.show()